In [1]:
import os
import pandas as pd
import numpy as np
import pyspark

import pyscisci.all as pyscisci
import time

path2mag = '/users/hgt6rn/Documents/DataSets/MAG'

mymag = pyscisci.MAG(path2mag, database_extension='hdf', show_progress=True)

In [2]:
pub2year = mymag.pub2year
pub2000 = np.sort([pid for pid,y in pub2year.items() if y==2000])
print(pub2000.shape)

(4279539,)


In [3]:
pub2ref = mymag.load_references(filter_dict={'CitedPublicationId':pub2000})


Loading pub2ref:   0%|          | 0/186 [00:00<?, ?it/s]

In [4]:
stime = time.time()

ncites = pub2ref.groupby('CitedPublicationId')['CitingPublicationId'].nunique()

etime = time.time()
print(etime-stime)

37.078683853149414


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct

spark = SparkSession.builder.config("spark.driver.memory", "50g").getOrCreate()

stime = time.time()
df = spark.createDataFrame(pub2ref)
print("Create df:", time.time()-stime)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/09 14:49:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/opt/homebrew/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/homebrew/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Create df: 402.3670480251312


In [6]:
stime = time.time()

df.groupby('CitedPublicationId').agg(countDistinct('CitingPublicationId')).show()

print("Compute Groupby:", time.time()-stime)

22/12/09 14:56:26 WARN TaskSetManager: Stage 0 contains a task of very large size (66074 KiB). The maximum recommended task size is 1000 KiB.


22/12/09 14:57:27 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
java.net.SocketException: Connection reset
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:186)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:252)
	at java.base/java.io.BufferedInputStream.read1(BufferedInputStream.java:292)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:351)
	at java.base/java.io.DataInputStream.readFully(DataInputStream.java:200)
	at java.base/java.io.DataInputStream.readFully(DataInputStream.java:170)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:758)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:747)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at 

Py4JJavaError: An error occurred while calling o56.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 0.0 failed 1 times, most recent failure: Lost task 2.0 in stage 0.0 (TID 2) (172.25.42.61 executor driver): java.net.SocketException: Broken pipe (Write failed)
	at java.base/java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.base/java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:110)
	at java.base/java.net.SocketOutputStream.write(SocketOutputStream.java:150)
	at java.base/java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:81)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:127)
	at java.base/java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.base/java.io.FilterOutputStream.write(FilterOutputStream.java:108)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:295)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(